In [203]:
import numpy as np # importing numpy for numerical computation
from sklearn.datasets import load_boston # here we are using sklearn's boston dataset
from sklearn.metrics import mean_squared_error # importing mean_squared_error metric
from sklearn.tree import DecisionTreeRegressor
import pickle
import pandas as pd
from joblib import dump, load

In [41]:
boston = load_boston()
x=boston.data #independent variables
y=boston.target #target variable
x.shape,y.shape

((506, 13), (506,))

In [177]:
def generating_samples(input_data, target_data):
    
    l=len(input_data)
    sample_index=np.random.choice(l,303,replace=False) # Sample 60 % of data form main data set(303)
    resample_index=np.random.choice(sample_index,203,replace=False) # Resample 40 % of main-data from sample data set (203)
    total_index=np.concatenate((sample_index,resample_index)) # joining sample and resample data
    data=np.array(input_data.iloc[total_index])
    col=np.sort(np.random.choice(13,(np.random.choice(np.arange(3,13))),replace=False)) # column sampling
    total_data=data[:,col]  #total_index[:,None] None part for changing the dim
    target_data_sd=target_data[total_index]
    #print(total_data.shape,target_data_sd.shape,)
    
    return total_data,target_data_sd,sample_index,col

In [147]:
def grader_samples(a,b,c,d):
    length = (len(a)==506  and len(b)==506)
    sampled = (len(a)-len(set([str(i) for i in a]))==203)
    rows_length = (len(c)==303)
    column_length= (len(d)>=3)
    assert(length and sampled and rows_length and column_length)
    return True
a,b,c,d = generating_samples(x, y)
grader_samples(a,b,c,d)

True

In [148]:
def sample_gen30(x,y):
    list_input_data = []
    list_output_data = []
    list_selected_row = []
    list_selected_columns = []


    for i in range(30):
        d,o,r,c=generating_samples(x, y)
        list_input_data.append(d)
        list_output_data.append(o)
        list_selected_row.append(r)
        list_selected_columns.append(c)
    
        
    return list_input_data,list_output_data,list_selected_row,list_selected_columns
list_input_data,list_output_data,list_selected_row,list_selected_columns= sample_gen30(x,y)

In [149]:
def grader_30(a):
    assert(len(a)==30 and len(a[0])==506)
    return True
grader_30(list_input_data)

True

#### step 2

In [179]:
def oob_30(list_selected_row,list_selected_columns):
    # Predicting on unseen data
    index_data_oob=[]                   #unseen data index
    for i in list_selected_row:
        s=np.unique(i)
        l1=list(np.arange(506))
        for j in s:
            l1.remove(j)
        index_data_oob.append(l1)
        
    data_oob=[]                    #unseen data from the index
    Y_oob=[]
    data_mse=[]
    Y_mse=[]
    for i in range(30): 
        temp=np.array(x)                          # converting input data into numpy array
        temp2=temp[index_data_oob[i],:]           # selecting the row which are not present in the sample data "temp2"
        temp3=temp2[:,list_selected_columns[i]]   # Selecting the columns from the temp2 for oob
        temp4=temp[:,list_selected_columns[i]     # Selecting the columns for mse from temp total data set 
        data_mse.append(temp4)
        data_oob.append(temp3)
        Y_mse.append(y)
        Y_oob.append(y[index_data_oob[i]])
    col_oob=list_selected_columns
    
    return data_oob,Y_oob,col_oob,data_mse,Y_mse

X_oob,Y_oob,col_oob,data_mse,Y_mse=oob_30(list_selected_row,list_selected_columns)

In [205]:
def model_train(list_X,list_Y,n):
    # https://scikit-learn.org/stable/modules/model_persistence.html

    DT=DecisionTreeRegressor(random_state=0,max_depth=12,min_samples_split=2)#random_state=0
    models=[]
    for i in range(n):
        x=np.array(list_X[i])
        y=np.array(list_Y[i])
        temp=list_X[i]
        temp2=list_Y[i]
        #print(list_X[i],list_Y[i])
        #print(x.shape,y.shape)
        clf=DT.fit(list_X[i],list_Y[i])
        models.append(pickle.dumps(clf))  
    return models

models=model_train(X_oob,Y_oob,30)

In [206]:
def model_pred(X,models,n):
    Y=[]
    for i in range(n):
        clf=pickle.loads(models[i])
        Y.append(clf.predict(X[i]))
    return Y
        

In [201]:
def mse(y_true,y_pred,n):
    mse=[]
    for i in range(n):
        mse.append(mean_squared_error(y_true[i],y_pred[i]))
    MSE=np.median(mse)
    return MSE

In [218]:
def MSE_OOB(x,y):
    list_input_data,list_output_data,list_selected_row,list_selected_columns=sample_gen30(x,y) # Creating  lists of sample
    X_oob,Y_oob,col_oob,X_mse,Y_mse=oob_30(list_selected_row,list_selected_columns)   # with index of sample list cearting data for test of oob
    models=model_train(list_input_data,list_output_data,30)  # training the model on sampled data 
    
    
    Y_pred=model_pred(X_mse,models,30)  # predicted the model on all the data that is unseen and seen 
    Y_pred_oob=model_pred(X_oob,models,30)  # predicting the output on unseen data by same model
    
    
    MSE=mse(Y_mse,Y_pred,30)  
    OOB=mse(Y_oob,Y_pred_oob,30)
    print(MSE,OOB)
    return MSE,OOB
MSE_OOB(x,y)

12.007369812331596 28.2003547858471


(12.007369812331596, 28.2003547858471)

## Observation Task 1
#### Mean square error of prediction of seen data with unseen data is low as compare to the Mean square error of prediction of unseen data, thus it is difficult to predict on unseen data by the model.
#### DecisionTreeRegressor or model is not immune to the outliers therefore we go with median of the answer.

## Task 2

In [216]:
list_MSE=[]
list_OOB=[]
lsc=[]
for i in range(35):
    m,o=MSE_OOB(x,y)
    list_MSE.append(m)
    list_OOB.append(o)

12.739719412100001 31.425825660445838
14.090846785329074 34.91124042768483
14.59792002058915 34.5213899421709
11.5759649984439 28.737044451160894
13.257707502070495 32.748277684003924
14.654715305011385 35.69394476826905
13.786197161106902 33.94313558241744
14.105890018105388 34.965013276628206
13.37105776733657 32.93094929627104
15.096465034924428 36.942334270496474
13.754091353669487 34.046745357336576
13.788029755563691 34.01504148537293
12.970182953884487 31.923311040544508
14.63143280705222 36.30782298185702
12.644280646891893 31.281586408225188
11.846350776312569 29.003259488316004
14.336575231183506 34.814218737225666
13.160456619872896 32.587795895253024
11.576949740568143 28.503373333802962
12.250439666588353 29.968066166595435
12.11556429454205 30.085965516257815
15.44912065455435 37.32921426970074
13.85953277961395 34.23412605608219
12.90742598203487 32.006069397335196
12.914460281299338 31.59032789896284
13.383779573154797 32.34960592682087
14.103864613542129 34.68971912818

In [182]:
def ci(data):#calculating the confidence interval
    data=np.array(data)
    mean=data.mean()
    std=data.std()
    size=len(data)
    left_limit = np.round(mean - 2*(std/np.sqrt(size)), 3) 
    right_limit = np.round(mean + 2*(std/np.sqrt(size)), 3)
    return left_limit,right_limit

In [193]:
left,right=ci(list_MSE)
print("Confidence Interval Of MSE :[{} ,{}]" .format(left,right))

Confidence Interval Of MSE :[13.587 ,14.468]


In [194]:
left,right=ci(list_OOB)
print("Confidence Interval Of MSE :[{} ,{}]" .format(left,right))

Confidence Interval Of MSE :[33.331 ,35.403]


## Observation task 2
#### Confidence Interval of MSE is 13.587 to 14.468 means 95% of the time our will be between these interval.
#### Confidence Interval of OOB is 33.331 to 35.403 means 95% of the time our will be  between these interval.


## Task 3

In [215]:
Xq=np.array([0.18,20.0,5.00,0.0,0.421,5.60,72.2,7.95,7.0,30.0,19.1,372.13,18.60])

list_input_data,list_output_data,list_selected_row,list_selected_columns=sample_gen30(x,y)
DT=DecisionTreeRegressor(random_state=0,max_depth=12,min_samples_split=2)

X_q=[]
op=[]
for i in range(30):
    DT.fit(list_input_data[i],list_output_data[i])
    op.append(DT.predict(Xq[list_selected_columns[i]].reshape(1,-1)))
    
print(np.median(op))



18.7


## Obervation Task 3
#### According CI confident interval of OOB data mean prediction on unseen data our prediction ”18.7” on this query point is +or- 5.9=sqrt(35) points off to an actual value.